In [1]:
#!/usr/bin/python3
# -*-coding:utf-8
'''
Created on Fri Dec 1 22:22:35 2017

@author: Ray


'''

import time
import pandas as pd
import numpy as np
from tqdm import tqdm
import os
import utils # written by author
from glob import glob
from datetime import datetime, timedelta
import multiprocessing as mp
import gc # for automatic releasing memory



In [2]:
##################################################
# Load  
##################################################
input_col = ['msno','transaction_date','membership_duration','is_membership_duration_equal_to_plan_days',
'is_membership_duration_longer_than_plan_days',]
membership_loyalty = utils.read_multiple_csv('../../input/preprocessed_data/transactions_date_base',input_col) # 20,000,000


100%|██████████| 4/4 [00:26<00:00,  6.56s/it]


In [3]:
membership_loyalty = membership_loyalty.head(n=500)


In [4]:
##################################################
# Convert string to datetime format
##################################################
membership_loyalty['transaction_date']  = membership_loyalty.transaction_date.apply(lambda x: datetime.strptime(x, '%Y-%m-%d'))


In [5]:
#==============================================================================
print('reduce memory')
#==============================================================================
utils.reduce_memory(membership_loyalty)


100%|██████████| 5/5 [00:00<00:00, 2624.72it/s]

reduce memory


In [6]:
membership_loyalty.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 500 entries, 0 to 499
Data columns (total 5 columns):
msno                                            500 non-null object
transaction_date                                500 non-null datetime64[ns]
membership_duration                             500 non-null int16
is_membership_duration_equal_to_plan_days       500 non-null int8
is_membership_duration_longer_than_plan_days    500 non-null int8
dtypes: datetime64[ns](1), int16(1), int8(2), object(1)
memory usage: 13.7+ KB


In [8]:
membership_loyalty.head()

,msno,transaction_date,membership_duration,is_membership_duration_equal_to_plan_days,is_membership_duration_longer_than_plan_days,days_longer_than_plan_days,is_early_expiration,early_expiration_days
0,+++FOrTS7ab3tIgIh8eWwX4FqRv8w/FoiOuyXsFvphY=,2016-09-09,5,0,0,0,0,0
1,+++IZseRRiQS9aaSkH6cMYU6bGDcxUieAi/tH67sC5s=,2015-11-21,410,1,0,0,0,0
2,+++IZseRRiQS9aaSkH6cMYU6bGDcxUieAi/tH67sC5s=,2016-10-23,471,0,1,76,0,0
3,+++hVY1rZox/33YtvDgmKA2Frg/2qhkz12B9ylCvh8o=,2016-11-16,29,0,0,0,0,0
4,+++hVY1rZox/33YtvDgmKA2Frg/2qhkz12B9ylCvh8o=,2016-12-15,31,0,1,1,0,0


In [10]:
T = 0
folder = 'trainW-'+str(T)
train = pd.read_csv('../../input/preprocessed_data/trainW-{0}.csv'.format(T))[['msno','is_churn']]

In [14]:
d = pd.merge(train,membership_loyalty, on = ['msno'], how = 'left')

In [53]:
df = pd.merge(train, 
        membership_loyalty[(membership_loyalty.transaction_date < datetime.strptime('2017-03-01', '%Y-%m-%d'))], 
        on=['msno'], 
        how='left')
df.head()

,msno,is_churn,transaction_date,membership_duration,is_membership_duration_equal_to_plan_days,is_membership_duration_longer_than_plan_days,days_longer_than_plan_days,is_early_expiration,early_expiration_days
0,zLo9f73nGGT1p21ltZC3ChiRnAVvgibMyazbCxvWPcg=,1,2015-08-19,30,1,0,0,0,0
1,zLo9f73nGGT1p21ltZC3ChiRnAVvgibMyazbCxvWPcg=,1,2015-09-19,30,1,0,0,0,0
2,zLo9f73nGGT1p21ltZC3ChiRnAVvgibMyazbCxvWPcg=,1,2015-10-23,30,1,0,0,0,0
3,zLo9f73nGGT1p21ltZC3ChiRnAVvgibMyazbCxvWPcg=,1,2015-11-22,30,1,0,0,0,0
4,zLo9f73nGGT1p21ltZC3ChiRnAVvgibMyazbCxvWPcg=,1,2015-12-20,32,0,1,2,0,0


In [78]:
e = df[df.msno == 'zLo9f73nGGT1p21ltZC3ChiRnAVvgibMyazbCxvWPcg='].membership_duration.cumsum()
e

30

In [57]:
df = df.head(n= 500)

In [90]:
tbl = df.groupby('msno').membership_duration.mean().to_frame()
tbl.columns = ['membership_duration-mean']
tbl['membership_duration-min'] = df.groupby('msno').membership_duration.min()
tbl['membership_duration-max'] = df.groupby('msno').membership_duration.max()
tbl['membership_duration-median'] = df.groupby('msno').membership_duration.median()
tbl['membership_duration-std'] = df.groupby('msno').membership_duration.std()
tbl['transactions_count'] = df.groupby('msno').membership_duration.size()
#tbl['membership_duration-cumsum'] = df.groupby('msno').membership_duration.cumsum()/tbl.transactions_count
# 'membership_duration-ratio'
#tbl['membership_duration-ratio'] = tbl.transactions_count/

tbl = tbl.reset_index()
tbl

,msno,membership_duration-mean,membership_duration-min,membership_duration-max,membership_duration-median,membership_duration-std,transactions_count
0,/66xO0tgKHeHrtaULlc3sOtEysp2v2y+kjcaqXScvyo=,37.000000,29,39,39.0,3.696846,7
1,/L2095JD4M/BNLTCboucgyuT2i8JIiJ87bOIsxtFvBw=,30.384615,28,31,31.0,0.897861,26
2,1AzXWFlRO6EfMBzfBdk98sBVnjzY7U1G24mVFNdzGNQ=,31.181818,29,32,31.0,0.906924,22
3,2OaV7/7vyAA2FrCkV4qXZEZYg5IzVyFNkQnQorgQ440=,35.333333,28,47,31.0,10.214369,3
4,8iF/+8HY8lJKFrTc7iR9ZYGCG2Ecrogbc2Vy5YhsfhQ=,435.137931,-657,762,547.0,361.910775,29
5,K6fja4+jmoZ5xG6BypqX80Uw/XKpMgrEMdG2edFOxnA=,182.258065,154,186,185.0,9.447990,31
6,L9W/dW7SdNQD2ve5egWKsXdBNKL0+3lFyG4FDXZMvpM=,30.384615,28,31,31.0,0.897861,26
7,NTOh3fAZrLoIyE1lxoRpp2lzEraNnnWbUvOhrmO1bUM=,238.277778,-490,549,273.5,218.894314,36
8,TotbNxPUifAoL9diWEMXcRnBgKDStI/a/SdXcf5WhfQ=,30.047619,30,31,30.0,0.218218,21
9,WkF/FvlzpBLFoa+Hm3hagOfme2pHLE2y+RJ3eGcLT0k=,27.111111,1,31,31.0,9.841804,9


In [77]:
tbl[tbl.msno == '/66xO0tgKHeHrtaULlc3sOtEysp2v2y+kjcaqXScvyo=']

,msno,membership_duration-mean,membership_duration-min,membership_duration-max,membership_duration-median,membership_duration-std,membership_duration-cumsum,transactions_count
0,/66xO0tgKHeHrtaULlc3sOtEysp2v2y+kjcaqXScvyo=,37.0,29,39,39.0,3.696846,NaN,7


In [94]:
df[df.msno == '2OaV7/7vyAA2FrCkV4qXZEZYg5IzVyFNkQnQorgQ440=']

,msno,is_churn,transaction_date,membership_duration,is_membership_duration_equal_to_plan_days,is_membership_duration_longer_than_plan_days,days_longer_than_plan_days,is_early_expiration,early_expiration_days
3,2OaV7/7vyAA2FrCkV4qXZEZYg5IzVyFNkQnQorgQ440=,1,2015-02-11,47,0,1,17,0,0


In [93]:
def near5(x, keep = 5):
    return x.tail(keep)
df = df.groupby('msno').apply(near5,1).reset_index(drop = True)
tbl = df.groupby('msno').membership_duration.mean().to_frame()
tbl.columns = ['membership_duration-mean']
tbl['membership_duration-min'] = df.groupby('msno').membership_duration.min()
tbl['membership_duration-max'] = df.groupby('msno').membership_duration.max()
tbl['membership_duration-median'] = df.groupby('msno').membership_duration.median()
tbl['membership_duration-std'] = df.groupby('msno').membership_duration.std()
tbl['transactions_count'] = df.groupby('msno').membership_duration.size()
#tbl['membership_duration-cumsum'] = df.groupby('msno').membership_duration.cumsum()/tbl.transactions_count
# 'membership_duration-ratio'
#tbl['membership_duration-ratio'] = tbl.transactions_count/

tbl = tbl.reset_index()
tbl

,msno,membership_duration-mean,membership_duration-min,membership_duration-max,membership_duration-median,membership_duration-std,transactions_count
0,/66xO0tgKHeHrtaULlc3sOtEysp2v2y+kjcaqXScvyo=,39,39,39,39,NaN,1
1,/L2095JD4M/BNLTCboucgyuT2i8JIiJ87bOIsxtFvBw=,28,28,28,28,NaN,1
2,1AzXWFlRO6EfMBzfBdk98sBVnjzY7U1G24mVFNdzGNQ=,32,32,32,32,NaN,1
3,2OaV7/7vyAA2FrCkV4qXZEZYg5IzVyFNkQnQorgQ440=,47,47,47,47,NaN,1
4,8iF/+8HY8lJKFrTc7iR9ZYGCG2Ecrogbc2Vy5YhsfhQ=,0,0,0,0,NaN,1
5,K6fja4+jmoZ5xG6BypqX80Uw/XKpMgrEMdG2edFOxnA=,183,183,183,183,NaN,1
6,L9W/dW7SdNQD2ve5egWKsXdBNKL0+3lFyG4FDXZMvpM=,28,28,28,28,NaN,1
7,NTOh3fAZrLoIyE1lxoRpp2lzEraNnnWbUvOhrmO1bUM=,0,0,0,0,NaN,1
8,TotbNxPUifAoL9diWEMXcRnBgKDStI/a/SdXcf5WhfQ=,31,31,31,31,NaN,1
9,WkF/FvlzpBLFoa+Hm3hagOfme2pHLE2y+RJ3eGcLT0k=,1,1,1,1,NaN,1


In [29]:
T = 0
folder = 'trainW-'+str(T)
train = pd.read_csv('../../input/preprocessed_data/trainW-{0}.csv'.format(T))[['msno','is_churn']]


In [35]:
e = pd.merge(train,pd.read_csv('../../feature/trainW-0/membership_duration.csv'), on = 'msno', how = 'left')
e.dropna()

,msno,is_churn,membership_duration-mean,membership_duration-min,membership_duration-max,membership_duration-median,membership_duration-std,transactions_count
58581,++1Wu2wKBA60W9F9sMh15RXmh1wN1fjoVGzNqvw/Gro=,0,30.222221,28.0,31.0,31.0,1.092906,9.0
86371,++4RuqBw0Ss6bQU4oMxaRlbBPoWzoEiIZaxPM04Y4+U=,0,30.384615,28.0,31.0,31.0,0.897861,26.0
164064,++/UDNo9DLrxT8QVGiDi1OnWfczAdEwThaVyD0fXO50=,0,53.099998,51.0,54.0,53.5,1.119210,20.0
255633,++3Z+W8OPnpbHYfrKwqRKN1bF83XEbxjdYUolhGdHZg=,0,31.518518,29.0,36.0,32.0,1.251779,27.0
384382,++4cUL0b9CfW8cj0A/wfSxQc4k4fcVtWcLqk2UOdpKs=,0,27.892857,-1.0,32.0,31.0,9.554287,28.0
400437,++4wuAZmfzMBjRHP4vDpTk+jkj9Xam8SW5rNJrGFEsE=,0,30.333334,28.0,31.0,31.0,1.000000,9.0
419150,++38dVm0EHKCOfND6oEal3xFKtqJYZTZJmnpbAOOZBk=,0,30.411764,28.0,31.0,31.0,0.870260,17.0
439563,++0BJXY8tpirgIhJR14LDM1pnaRosjD1mdO1mIKxlJA=,0,36.250000,30.0,120.0,30.0,22.472204,16.0
586237,++/9R3sX37CjxbY/AaGvbwr3QkwElKBCtSvVzhCBDOk=,0,30.416666,28.0,31.0,31.0,0.900337,12.0
623009,++0wqjjQge1mBBe5r4ciHGKwtF/m322zkra7CK8I+Mw=,0,30.312500,28.0,31.0,31.0,0.946485,16.0


In [6]:
def near5(x, keep = 5):
    return x.tail(keep)
#==============================================================================
# def
#==============================================================================
def make(T):
	"""
	T = 0
	folder = 'trainW-0'
	"""

	if T==-1:
	    folder = 'test'
	    train = pd.read_csv('../input/sample_submission_v2.csv') # 此train代表的是test的user
	else:
	    folder = 'trainW-'+str(T)
	    train = pd.read_csv('../input/preprocessed_data/trainW-{0}.csv'.format(T))[['msno','is_churn']]
	##################################################
	# All history
	##################################################
	# merge
	df = pd.merge(train, 
	    membership_loyalty[membership_loyalty.w == T], 
	    on=['msno','w'], 
	    how='left')
	# core
	col =['msno','days_since_the_last_expiration-mean', 'days_since_the_last_expiration-std',
	 'days_since_the_last_expiration-min','days_since_the_last_expiration-max',
	 'days_since_the_last_expiration-50%']

	df = df.dropna()

	df = df.groupby('msno').days_since_the_last_expiration.describe().add_prefix('days_since_the_last_expiration-').reset_index(level=0)[col]
	# write
	df.to_csv('../feature/{}/days_since_the_last_expiration.csv'.format(folder), index = False)
	##################################################
	# near 5
	##################################################
	# merge
	df = pd.merge(train, 
	    membership_loyalty[(membership_loyalty.w == T) & (membership_loyalty.order_number_rev <=5)], 
	    on=['msno','w'], 
	    how='left')
	# core
	col =['msno','days_since_the_last_expiration_n5-mean', 'days_since_the_last_expiration_n5-std',
	 'days_since_the_last_expiration_n5-min','days_since_the_last_expiration_n5-max',
	 'days_since_the_last_expiration_n5-50%']

	df = df.dropna()

	df = df.groupby('msno').days_since_the_last_expiration.describe().add_prefix('days_since_the_last_expiration_n5-').reset_index(level=0)[col]
	# write
	df.to_csv('../feature/{}/days_since_the_last_expiration_n5.csv'.format(folder), index = False)

	##################################################
	# only one prvious order
	##################################################
	# merge
	df = pd.merge(train, 
	    membership_loyalty[(membership_loyalty.w == T) & (membership_loyalty.order_number_rev == 1)], 
	    on=['msno','w'], 
	    how='left')
	# core
	col =['msno','days_since_the_last_expiration_n5-mean', 'days_since_the_last_expiration_n5-std',
	 'days_since_the_last_expiration_n5-min','days_since_the_last_expiration_n5-max',
	 'days_since_the_last_expiration_n5-50%']

	df = df.dropna()

	df = df.groupby('msno').days_since_the_last_expiration.describe().add_prefix('days_since_the_last_expiration_n5-').reset_index(level=0)[col]
	# write
	df.to_csv('../feature/{}/days_since_the_last_expiration_n1.csv'.format(folder), index = False)
	del df
	del train
	gc.collect()
	print ('{0} done'.format(T))


In [25]:
def reduce_memory(df, ix_start=0):
	# There are two concepts: 1. change type, 2 drop NaN(missing) value
    df.fillna(-1, inplace=True)
    if df.shape[0] <= 500:
        df_ = df # sampling for the following comoparison
    else:
        print (df.shape)
        #df_ = df.sample(500, random_state=71)
    ## int
    col_int8 = []
    col_int16 = []
    col_int32 = []
    for c in tqdm(df.columns[ix_start:], miniters=20):
        if df[c].dtype =='O':
        	# '0': Objects
            continue
        elif df[c].dtype == 'datetime64[ns]':
            continue
        elif (df_[c] == df_[c].astype(np.int8)).all():
        	# Series.all():return true if all elements in Series is True
            col_int8.append(c)
        elif (df_[c] == df_[c].astype(np.int16)).all():
            col_int16.append(c)
        elif (df_[c] == df_[c].astype(np.int32)).all():
            col_int32.append(c)
    
    df[col_int8]  = df[col_int8].astype(np.int8)
    df[col_int16] = df[col_int16].astype(np.int16)
    df[col_int32] = df[col_int32].astype(np.int32)
    
    ## float
    col = [c for c in df.dtypes[df.dtypes==np.float64].index]
    df[col] = df[col].astype(np.float32)

    gc.collect()



In [26]:
def near(x, keep = 5):
    return x.tail(keep)
#==============================================================================
# def
#==============================================================================
def make(T):
    """
    T = 0
    folder = 'trainW-0'
    """

    if T ==-1:
        folder = 'test'
        train = pd.read_csv('../../input/sample_submission_v2.csv') # 此train代表的是test的user
    else:
        folder = 'trainW-'+str(T)
        train = pd.read_csv('../../input/preprocessed_data/trainW-{0}.csv'.format(T))[['msno','is_churn']]

    # the following style is silly, but it's all for saving memory 
    if T == 0:
        df = pd.merge(train, 
        membership_loyalty[(membership_loyalty.transaction_date < datetime.strptime('2017-03-01', '%Y-%m-%d'))], 
        on=['msno'], 
        how='left')
        del train
    elif T == 1:
        # w = 1:使用2月之前的資料當作history
        df = pd.merge(train, 
        membership_loyalty[(membership_loyalty.transaction_date < datetime.strptime('2017-02-01', '%Y-%m-%d'))],
        on=['msno'], 
        how='left') 
        del train
    elif T == 2:
        # w = 2:使用1月之前的資料當作history
        df = pd.merge(train, 
        membership_loyalty[(membership_loyalty.transaction_date < datetime.strptime('2017-01-01', '%Y-%m-%d'))],
        on=['msno'], 
        how='left') 
        del train
    elif T == -1:
        # w = -1:使用4月之前的資料當作history
        df = pd.merge(train, 
        membership_loyalty[(membership_loyalty.transaction_date < datetime.strptime('2017-04-01', '%Y-%m-%d'))],
        on='msno', 
        how='left') 
        del train
    ##################################################
    # All history
    ##################################################

    #core
    tbl = df.groupby('msno').membership_duration.mean().to_frame()
    tbl.columns = ['membership_duration-mean']
    tbl['membership_duration-min'] = df.groupby('msno').membership_duration.min()
    tbl['membership_duration-max'] = df.groupby('msno').membership_duration.max()
    tbl['membership_duration-median'] = df.groupby('msno').membership_duration.median()
    tbl['membership_duration-std'] = df.groupby('msno').membership_duration.std()
    tbl['transactions_count'] = df.groupby('msno').membership_duration.size()
    #==============================================================================
    print('reduce memory')
    #==============================================================================
    utils.reduce_memory(tbl)
    #write
    tbl.reset_index().to_csv('../../feature/{}/membership_duration.csv'.format(folder), index = False)
    del tbl
    gc.collect()
    ##################################################
    # near 5
    ##################################################
    #core
    print ('1')
    df = df.dropna()
    df_ = df.groupby('msno').apply(near,5).reset_index(drop = True)
    print ('2')
    tbl = df_.groupby('msno').membership_duration.mean().to_frame()
    tbl.columns = ['membership_duration-mean_n5']
    tbl['membership_duration-min_n5'] = df_.groupby('msno').membership_duration.min()
    tbl['membership_duration-max_n5'] = df_.groupby('msno').membership_duration.max()
    tbl['membership_duration-median_n5'] = df_.groupby('msno').membership_duration.median()
    tbl['membership_duration-std_n5'] = df_.groupby('msno').membership_duration.std()
    tbl.reset_index(inplace = True)
    #==============================================================================
    print('reduce memory')
    #==============================================================================
    reduce_memory(tbl)

    #write
    tbl.to_csv('../../feature/{}/membership_duration_n5.csv'.format(folder), index = False)
    del tbl
    gc.collect()
    ##################################################
    # only one prvious order
    ##################################################
    #core
    tbl = df.groupby('msno').apply(near,1).reset_index(drop = True).groupby('msno').membership_duration.mean().to_frame()
    tbl.columns = ['membership_duration-mean_n1']
    tbl['membership_duration-min_n1'] = df.groupby('msno').membership_duration.min()
    tbl['membership_duration-max_n1'] = df.groupby('msno').membership_duration.max()
    tbl['membership_duration-median_n1'] = df.groupby('msno').membership_duration.median()
    tbl['membership_duration-std_n1'] = df.groupby('msno').membership_duration.std()
    #==============================================================================
    print('reduce memory')
    #==============================================================================
    utils.reduce_memory(tbl)
    #write
    tbl.reset_index().to_csv('../../feature/{}/membership_duration_n1.csv'.format(folder), index = False)
    del tbl
    gc.collect()


In [27]:
s = time.time()
make(0)
e = time.time()
e-s

100%|██████████| 6/6 [00:00<00:00, 2575.30it/s]

reduce memory



100%|██████████| 6/6 [00:00<00:00, 2533.81it/s]

1
2
reduce memory
reduce memory


ValueError: Cannot take a larger sample than population when 'replace=False'